In [3]:
from bs4 import BeautifulSoup #スクレイピングのためのライブラリ
import requests #HTTP操作用
import re
import time
import pandas as pd
import numpy as np
import sqlite3

/Users/takahashikaisei/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
y = 2023
m = 12
url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=44&block_no=0370&year={y}&month={m}'
r = requests.get(url)

In [5]:
r

<Response [200]>

In [17]:
soup = BeautifulSoup(r.content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [7]:
print(soup.find('title').string)

気象庁｜過去の気象データ検索


classをdata_0_0で絞ればとりたいものはとれる

In [31]:
tags2 = soup.find('td',class_ = 'data_0_0').string
tags2

'0.0'

これをfor文で回して、データベースにいい感じに格納すればいい

for文で全部を取得できるようにする

In [29]:
tags3 = soup.find_all('td',class_ = 'data_0_0')

data = []
for i in tags3:
    data.append(i.string)


In [30]:
data

['0.0',
 '0.0',
 '0.0',
 '9.0',
 '13.2',
 '5.6',
 '///',
 '///',
 '2.2',
 '4.1',
 '南東',
 '6.6',
 '北',
 '北',
 '6.0',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '8.5',
 '13.2',
 '4.9',
 '///',
 '///',
 '2.1',
 '3.3',
 '北',
 '6.2',
 '北西',
 '北',
 '9.1',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '9.7',
 '16.0',
 '4.9',
 '///',
 '///',
 '3.0',
 '8.5',
 '南西',
 '10.7',
 '南西',
 '北北西',
 '9.2',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '9.6',
 '14.2',
 '5.6',
 '///',
 '///',
 '2.2',
 '4.1',
 '北北西',
 '7.3',
 '北西',
 '北北西',
 '9.2',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '8.4',
 '10.4',
 '6.1',
 '///',
 '///',
 '2.4',
 '3.6',
 '北北西',
 '7.3',
 '北北西',
 '北北西',
 '0.0',
 '///',
 '///',
 '0.5',
 '0.5',
 '0.5',
 '12.1',
 '16.4',
 '8.1',
 '///',
 '///',
 '2.6',
 '4.8',
 '南東',
 '7.6',
 '北北東',
 '北',
 '6.7',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '14.0',
 '19.3',
 '9.1',
 '///',
 '///',
 '5.0',
 '15.3',
 '南西',
 '19.0',
 '南西',
 '北北西',
 '8.6 )',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '13.2',
 '18.3',
 '6.1',
 '

リストに格納は嫌だからデータフレームに格納したい

In [80]:
tags3 = soup.find_all('td', class_='data_0_0')

data = []
for i in tags3:
    data.append(i.string)

final_list = []
day_num = 31
final_list = np.array_split(data, day_num) #31日だから31ずつに区切ればいい

# データフレームに変換する
df = pd.DataFrame(final_list, columns=[
    "合計降水量",
    "最大一時間降水量",
    "最大10分間降水量",
    "平均気温",
    "最高気温",
    "最低気温",
    "平均湿度",
    "最小湿度",
    "平均風速",
    "最大風速",
    "最大風速の風向き",
    "最大瞬間風速",
    "最大瞬間風速の風向き",
    "最多風向き",
    "日照時間",
    "降雪の深さの合計",
    "最深積雪"
    ])

df

,合計降水量,最大一時間降水量,最大10分間降水量,平均気温,最高気温,最低気温,平均湿度,最小湿度,平均風速,最大風速,最大風速の風向き,最大瞬間風速,最大瞬間風速の風向き,最多風向き,日照時間,降雪の深さの合計,最深積雪
0,0.0,0.0,0.0,9.0,13.2,5.6,///,///,2.2,4.1,南東,6.6,北,北,6.0,///,///
1,0.0,0.0,0.0,8.5,13.2,4.9,///,///,2.1,3.3,北,6.2,北西,北,9.1,///,///
2,0.0,0.0,0.0,9.7,16.0,4.9,///,///,3.0,8.5,南西,10.7,南西,北北西,9.2,///,///
3,0.0,0.0,0.0,9.6,14.2,5.6,///,///,2.2,4.1,北北西,7.3,北西,北北西,9.2,///,///
4,0.0,0.0,0.0,8.4,10.4,6.1,///,///,2.4,3.6,北北西,7.3,北北西,北北西,0.0,///,///
5,0.5,0.5,0.5,12.1,16.4,8.1,///,///,2.6,4.8,南東,7.6,北北東,北,6.7,///,///
6,0.0,0.0,0.0,14.0,19.3,9.1,///,///,5.0,15.3,南西,19.0,南西,北北西,8.6 ),///,///
7,0.0,0.0,0.0,13.2,18.3,6.1,///,///,2.7,5.1,南南西,10.3,北北西,南西,8.4,///,///
8,0.0,0.0,0.0,12.6,17.4,6.9,///,///,1.4,2.6,南,4.5,北北西,北,8.7,///,///
9,0.0,0.0,0.0,13.2,17.5,9.3,///,///,1.6,3.0,北,5.2,北,北北西,9.1,///,///


In [83]:
month = 12
day = []
for i in range(1, day_num+1):
    day.append(i)

hiduke = []
for tasu in day:
    hiduke.append(str(month) + '/' + str(tasu))

s = pd.Series(hiduke, name='date')

df.insert(0, '日付', s)

リストに格納できるのはできる
for文の回し方はわかったから次はデータベースへの格納を考えよう

データベースへの格納を考えるよ
調べたら意外と簡単に変換できた。

In [87]:
con = sqlite3.connect('weather_scrape.sqlite') #データベースにアクセス

df.to_sql('weather_scrape_data', con, if_exists='replace', index=False) #データベースに変換、if_exists='replace'で同じ名前のものがあったときに置き換える。index=Falseでindexを入れないように指定してる

con.close() #データベースを閉じる

さっき作った余計なデータベースを削除したコード、一応コメントアウトにしとく

In [88]:
# #データベースを削除する
# con = sqlite3.connect('weather_scrape.sqlite')
# cur = con.cursor()

# cur.execute('DROP TABLE IF EXISTS weather_scrape_12month_data')
# con.commit()
# con.close()